# Dataset Preprocessing Sweep
Trying to set up a new template for a notebook that will run a simple 5-fold cross-validation XGBoost model on a variety of dataset permutations.

# Setup

In [1]:
# two manual flags (ex-config)
COLAB = False
USE_GPU = True
libraries = ['xgboost', 'lightgbm', 'catboost', 'widedeep-SAINT']

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"dataset_sweep_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [5]:
# handle Google Colab-specific library installation/updating
if COLAB:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
#     !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # upgrade sklearn
    !pip install --upgrade scikit-learn

#     !pip install category_encoders
    
    if 'catboost' in libraries:
        !pip install catboost
    
    if 'xgboost' in libraries:
        if USE_GPU: 
            # this part is from https://github.com/rapidsai/gputreeshap/issues/24
            !pip install cmake --upgrade
            # !pip install sklearn --upgrade
            !git clone --recursive https://github.com/dmlc/xgboost
            %cd /content/xgboost
            !mkdir build
            %cd build
            !cmake .. -DUSE_CUDA=ON
            !make -j4
            %cd /content/xgboost/python-package
            !python setup.py install --use-cuda --use-nccl
            !/opt/bin/nvidia-smi
            !pip install shap
        else:
            !pip install --upgrade xgboost
    if 'lightgbm' in libraries:
        if USE_GPU:
            # lighgbm gpu compatible
            !git clone --recursive https://github.com/Microsoft/LightGBM
            ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
        else:
            !pip install --upgrade lightgbm
        

        

Now, non-stdlib imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
from optuna.samplers import TPESampler
from sklearn.utils import resample
import seaborn as sns

# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler#, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import umap

# from boruta import BorutaPy
# from BorutaShap import BorutaShap
# import category_encoders as ce

In [29]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/nov2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/')
    datapath = root/'datasets'
    # edapath = root/'EDA'
    # modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'studies'
    
    for pth in [datapath, predpath, subpath, studypath]:
        pth.mkdir(exist_ok=True)
    


In [8]:
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=SEED)

## Ex-Model Config

In [9]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
#     'random_state': SEED,
#     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
#     'features_created': False,
#     'feature_creator': None,
}

## Data Loading

In [10]:
# loading the full training data set to get the feature correlations to target, for K-means clustering later
df = pd.read_feather(datapath/'train.feather')
# df_corr = df.corr() # getting the correlations of the features
# corr_target = df_corr.loc['target':'target'] # pulling out just the correlation of features with the target, as a 1-row df (for Series, it'd be df_corr.loc['target'])
corr_target = load(datapath/'feature_correlations_with_target.joblib')

In [11]:
corr_target.head()

f34    0.135270
f55    0.113889
f43    0.109418
f71    0.107687
f80    0.106964
Name: target, dtype: float64

In [12]:
# corr_target_x = corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
# corr_target_abs = abs(corr_target_x) # just interested in magnitudes here
# corr_sorted = corr_target_abs.sort_values(by='target', axis=1, ascending=False) # df columns of useful values by correlation with target, will be modified later
y = df.target # pulling out the dependent variable
X = df.drop('target', axis=1) # isolating the independent variables
del df # cleaning up memory
# categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # there aren't any

## Parameters for Baseline Model

In [13]:
# optuna 20211004, thru 106 trials on unaltered original dataset
# params = {
#     'n_estimators': 3878,
#     'max_depth': 4,
#     'learning_rate': 0.024785857161974977,
#     'reg_alpha': 26.867682044658245,
#     'reg_lambda': 10.839759074147148,
#     'subsample': 0.8208581489835881,
#     'min_child_weight': 8.829122644339664,
#     'colsample_bytree': 0.906420714280384,
#     'gamma': 1.472322916021486
# }

# Feature Selection Preload

In [14]:
# b = load(altdatapath/'X_boruta_200iter_filtered_green.joblib')
# type(b)

In [15]:
# bdf = pd.DataFrame(b, index=X.index).join(y)
# bdf.head()

(Following cells generate the correlations for the different feature selections, so that the process need not be repeated each iteration.)

In [16]:
# b = load(altdatapath/'X_boruta_200iter_filtered_green.joblib')
# bdf = pd.DataFrame(b, index=X.index).join(y)
# bdf.head()
# bdf_corr = bdf.corr()
# bdf_corr_target = bdf_corr.loc['target':'target']
# bdf_corr_target_x = bdf_corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
# bdf_corr_target_abs = abs(bdf_corr_target_x) # just interested in magnitudes here
# bdf_corr_sorted = bdf_corr_target_abs.sort_values(by='target', axis=1, ascending=False)
# dump(bdf_corr_sorted, altdatapath/'X_boruta_200iter_filtered_green_corr_sorted.joblib')

In [17]:
# del b, bdf, bdf_corr_target, bdf_corr_target_x, bdf_corr_target_abs, bdf_corr_sorted

In [18]:
# b = pd.read_feather(altdatapath/'X_boruta_shap_200trials.feather')
# bdf = b.join(y)
# # bdf.head()
# bdf_corr = bdf.corr()
# bdf_corr_target = bdf_corr.loc['target':'target']
# bdf_corr_target_x = bdf_corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
# bdf_corr_target_abs = abs(bdf_corr_target_x) # just interested in magnitudes here
# bdf_corr_sorted = bdf_corr_target_abs.sort_values(by='target', axis=1, ascending=False)
# dump(bdf_corr_sorted, altdatapath/'X_boruta_shap_200trials_corr_sorted.joblib')

In [22]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial, X=X, y=y): # categoricals=categoricals, corr_sorted=corr_sorted):

#     dump(pca60, edapath/'PCA_60.joblibg')

    # use the original 286-feature dataset, or the 136-feature BorutaShap selected one
#     dataset = trial.suggest_categorical('dataset', ['X_orig.feather', 'X_boruta_shap_200trials.feather']) 
#     train_source = altdatapath/'X_orig.feather'
    # train_source = altdatapath/'train-WITH-KMeans_12cluster_kmeans++_maxiter1000_rs42.feather' #'X_boruta_shap_200trials.feather'
#     X = pd.read_feather(path=train_source)
#     y = load(datapath/'y.joblib')
    
#     # decides whether binary-encoded categoricals are encoded or not
#     cardinality_min = trial.suggest_categorical('cardinality_min', [0, 2]) 
    
    # scaling
    scaler_type = trial.suggest_categorical('scaler', [StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler])
                                       
    # ------------------
    
#     # encoding
#     encoder_name = trial.suggest_categorical('encoder_name', ['woe', 'catboost', 'james-stein', 'loo', 'mestimate', 'target', 'hashing', None])
#     if encoder_name:
#         encode_before_kmeans = trial.suggest_categorical('encode_before_kmeans', [True, False]) # determines order
    
#     # define dict of encoders, with names as keys and implementations as values
#     encoders = {
#         'woe': ce.WOEEncoder(cols=categoricals),
#         'catboost': ce.CatBoostEncoder(cols=categoricals),
#         'james-stein': ce.JamesSteinEncoder(cols=categoricals),
#         'loo': ce.LeaveOneOutEncoder(cols=categoricals),
#         'mestimate': ce.MEstimateEncoder(cols=categoricals),
#         'target': ce.TargetEncoder(cols=categoricals),
#         'hashing': ce.HashingEncoder(cols=categoricals),
#     }
    
#     # ----------------
    
#     # feature selection setup -- applied before preprocessing
#     feature_selection = trial.suggest_categorical('feature_selection', ['BorutaShap', 'Boruta', None])
#     k_means_method = trial.suggest_categorical('k_means_method', [25, 50, 100, 'k-means++', None]) # K-Means initialization method
    
#     # now, switch datasets if feature selection is implemented; regardless, prepare appropriate K-Means setup (to be implemented later, in folds)
#     if feature_selection: # create a subset of features if appropriate
#         if feature_selection == 'BorutaShap':
#             X = pd.read_feather(altdatapath/'X_boruta_shap_200trials.feather') # :: pd.DataFrame
#             categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars
#             # k-means cluster feature generation setup 
#             if k_means_method:
#                 corr_sorted = load(altdatapath/'X_boruta_shap_200trials_corr_sorted.joblib') # load prepared correlations
#                 k_means_clusters = trial.suggest_int('k_means_clusters', 6, 12) # for grabbing the most useful features from `corr_sorted`
#                 useful_features = list(corr_sorted.columns[:k_means_clusters])
#         elif feature_selection == 'Boruta':
#             X = pd.DataFrame(load(altdatapath/'X_boruta_200iter_filtered_green.joblib'), index=X.index)
#             if k_means_method:
#                 corr_sorted = load(altdatapath/'X_boruta_200iter_filtered_green_corr_sorted.joblib') # load prepared correlations
#                 k_means_clusters = trial.suggest_int('k_means_clusters', 6, 12) # for grabbing the most useful features from `corr_sorted`
#                 useful_features = list(corr_sorted.columns[:k_means_clusters])
#             categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars
#     else:
#         if k_means_method:
#             k_means_clusters = trial.suggest_int('k_means_clusters', 6, 12) # for grabbing the most useful features from `corr_sorted`
#             useful_features = list(corr_sorted.columns[:k_means_clusters])
    
#     # -------------------------
    
#     # PCA dimensionality reduction setup -- applied at end of preprocessing
#     pca_components = trial.suggest_categorical('pca_components', [50, 75, 'mle', None, 'NO'])
    
    # define k-fold splitter
    kfold = KFold(n_splits=5, shuffle=False)
        
    # initialize lists for out-of-fold preds and ground truth
    oof_preds, oof_y = [], []
            
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
        print(f"FOLD {fold}")
        print("---------------------------------------------------")
        
        y_train, y_valid = y[train_ids], y[valid_ids] # slicing syntax works on both pandas.Series and numpy.ndarray
        # category_encoders expects pandas.DataFrames
        X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:] # bc need pandas.DataFrames for slicing
        
        
        # now, apply preprocessing
        scaler = scaler_type()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)
    
        
#         if encoder_name: # if categorical encoding to be applied to high cardinality (2<x<100,000) categoricals...
#             if k_means_method: # if k-means proceeding
#                 if encode_before_kmeans: # do category encoding, then clustering
#                     # category encoding for high-cardinality categoricals
#                     encoder = encoders[encoder_name]
#                     X_train = encoder.fit_transform(X_train, y_train)
#                     X_valid = encoder.transform(X_valid)

#                     # k-means cluster feature generation
#                     cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
#                     if k_means_method == 'k-means++':
#                         kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
#                     else:
#                         kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
#                     # fit on the training set only
#                     X_train_clusters = kmeans.fit_transform(X_train[useful_features])
#                     X_valid_clusters = kmeans.transform(X_valid[useful_features])
#                     # convert numpy.ndarrays back to properly-labeled pandas.DataFrames
#                     X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
#                     X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
#                     # join the cluster-distance features to the training and validation sets
#                     X_train = X_train.join(X_train_clusters)
#                     X_valid = X_valid.join(X_valid_clusters)

#                 else: # do k-means clustering, then do category encoding
#                     cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
#                     if k_means_method == 'k-means++':
#                         kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
#                     else:
#                         kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
#                     X_train_clusters = kmeans.fit_transform(X_train[useful_features])
#                     X_valid_clusters = kmeans.transform(X_valid[useful_features])
#                     X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
#                     X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
#                     X_train = X_train.join(X_train_clusters)
#                     X_valid = X_valid.join(X_valid_clusters)

#                     encoder = encoders[encoder_name]
#                     X_train = encoder.fit_transform(X_train, y_train)
#                     X_valid = encoder.transform(X_valid)
            
#             else: # category encoding, but no k-means
#                 encoder = encoders[encoder_name]
#                 X_train = encoder.fit_transform(X_train, y_train)
#                 X_valid = encoder.transform(X_valid)
                
#         else: # no category encoding
#             if k_means_method: # if still doing k-means
#                 # k-means cluster feature generation
#                 cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
#                 if k_means_method == 'k-means++':
#                     kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
#                 else:
#                     kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
#                 # fit on the training set only
#                 X_train_clusters = kmeans.fit_transform(X_train[useful_features])
#                 X_valid_clusters = kmeans.transform(X_valid[useful_features])
#                 # convert numpy.ndarrays back to properly-labeled pandas.DataFrames
#                 X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
#                 X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
#                 # join the cluster-distance features to the training and validation sets
#                 X_train = X_train.join(X_train_clusters)
#                 X_valid = X_valid.join(X_valid_clusters)
            
        
#         # now, PCA dimensionality reduction
#         if pca_components != 'NO':
#             pca = PCA(n_components=pca_components, random_state=42)
#             X_train = pca.fit_transform(X_train)
#             X_valid = pca.transform(X_valid)
            
        # define models
        model = XGBClassifier(
            booster='gbtree',
            tree_method='gpu_hist',
            random_state=42,
            # n_jobs=-1, 
            verbosity=1, 
            objective='binary:logistic',
            # **params
        )
        model.fit(X_train, y_train)
        y_valid_preds = model.predict_proba(X_valid)[:,1]

        # add the fold-model's OOF preds and ground truths to the out-of-loop lists
        oof_preds.extend(y_valid_preds)
        oof_y.extend(y_valid)


        fold_valid_auc = roc_auc_score(y_valid, y_valid_preds)
        print(f"Valid AUC for fold {fold} is {fold_valid_auc}")   
#         dump(model, Path(runpath/f"{library}_fold{fold}_rs{random_state}_model.joblib"))

    model_valid_auc = roc_auc_score(oof_y, oof_preds)
    print(f"Valid AUC score for is {model_valid_auc}")
    
    return model_valid_auc

In [23]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "arch": 'xgboost',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
#     "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
#     "scale_b4_impute": False,
#     "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
#     'optuna_trials': 50,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202111_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for preprocessing techniques on dataset -- scaling only",
    'config': exmodel_config,
}

In [24]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dataset_sweep_20211102.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)


In [27]:
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=int(SEED)), study_name='dataset_20211102')
# study = load()

[I 2021-11-02 18:43:05,558] A new study created in memory with name: dataset_20211102


In [28]:
for x in range(1, 5):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())


FOLD 0
---------------------------------------------------
[18:43:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.7201557997326132
FOLD 1
---------------------------------------------------
[18:43:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.7216802468693998
FOLD 2
---------------------------------------------------
[18:43:19] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore

[I 2021-11-02 18:43:25,499] Trial 0 finished with value: 0.7240968345002873 and parameters: {'scaler': <class 'sklearn.preprocessing._data.MinMaxScaler'>}. Best is trial 0 with value: 0.7240968345002873.


Valid AUC score for is 0.7240968345002873
FOLD 0
---------------------------------------------------
[18:43:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.7204444230609365
FOLD 1
---------------------------------------------------
[18:43:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.721680558581767
FOLD 2
---------------------------------------------------
[18:43:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly

[I 2021-11-02 18:43:41,301] Trial 1 finished with value: 0.7242674437944308 and parameters: {'scaler': <class 'sklearn.preprocessing._data.RobustScaler'>}. Best is trial 1 with value: 0.7242674437944308.


Valid AUC score for is 0.7242674437944308
FOLD 0
---------------------------------------------------
[18:43:43] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.7204444230609365
FOLD 1
---------------------------------------------------
[18:43:46] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.721680558581767
FOLD 2
---------------------------------------------------
[18:43:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly

[I 2021-11-02 18:43:57,059] Trial 2 finished with value: 0.7242674437944308 and parameters: {'scaler': <class 'sklearn.preprocessing._data.RobustScaler'>}. Best is trial 1 with value: 0.7242674437944308.


Valid AUC score for is 0.7242674437944308
FOLD 0
---------------------------------------------------
[18:43:57] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.7204444230609365
FOLD 1
---------------------------------------------------
[18:44:00] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.721680558581767
FOLD 2
---------------------------------------------------
[18:44:02] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly

[I 2021-11-02 18:44:08,896] Trial 3 finished with value: 0.7242674437944308 and parameters: {'scaler': <class 'sklearn.preprocessing._data.StandardScaler'>}. Best is trial 1 with value: 0.7242674437944308.


Valid AUC score for is 0.7242674437944308


NameError: name 'studypath' is not defined

In [30]:
dump(study, filename=studypath/f'optuna-dataset_study-scaling_only-trial5_20211102.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/studies/optuna-dataset_study-scaling_only-trial5_20211102.joblib']

In [31]:
study.best_trial.params

{'scaler': sklearn.preprocessing._data.RobustScaler}

In [32]:
wandb.log({'best_dataset_params': study.best_trial.params})
wandb.finish()

value,▁███
scaler,sklearn.preprocessin...
value,0.72427


So, the best of the sweep was trial 6 with AUC of 0.8529520938646249, but the straight-up analysis without any bells and whistles gets 0.8566651115202035. 

Conclusion: best to forget about preprocessing. $\blacksquare$